In [6]:
import requests
import json
import pandas as  pd
import time
# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

In [7]:
url = 'http://127.0.0.1:5000/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [8]:
response.json()


{'signingKey_hex': '33b0507ced2be6fdd92d81ad6d3552a791ecb5684f221a3c1d178735153a885e',
 'traderId': '1',
 'verifyKey_hex': '75be1702fc8ea93d434e661ce5c683bfa9d0ff8f9d21814b0a36e62ef818f8b3'}

In [9]:
url = 'http://127.0.0.1:5000/createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [10]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 1,
 'marketMin': 0,
 'marketRootId': 1,
 'traderId': '1'}

In [11]:
tId

'1'

In [12]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,1,1


In [14]:
# Create a trade
url = 'http://127.0.0.1:5000/createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.49, "quantity":1}

response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [15]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.49,
 'quantity': 1,
 'traderId': '1'}

In [ ]:
# Create a matched trade
url = 'http://127.0.0.1:5000/createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [37]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.49,
 'quantity': -1,
 'traderId': '1'}

In [13]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


NameError: name 'content_maketrade' is not defined

In [39]:
url = 'http://127.0.0.1:5000/createMarket'

for i in range(3):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 0.14838814735412598 seconds.
market created: 2 in 0.16618704795837402 seconds.
market created: 3 in 0.2869398593902588 seconds.


In [40]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 3,
 'traderId': 1}

In [41]:
url = 'http://127.0.0.1:5000/viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [42]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,0.7
1,2,1,0,0.7
2,3,1,0,0.7


In [43]:
url = 'http://127.0.0.1:5000/viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [44]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,tradeRootId,traderId
0,1,1,0.49,1,1,1
1,1,1,0.49,-1,2,1
10,1,1,0.49,1,3,1
11,1,1,0.49,-1,3,1
12,1,1,0.49,-1,5,1
13,1,1,0.49,1,6,1
14,1,1,0.49,-1,6,1
15,1,1,0.49,1,6,1
16,1,1,0.49,1,5,1
17,1,1,0.49,-1,5,1
